In [19]:
import os
import redis
import sys
import datetime
import pandas as pd

In [20]:
REDIS_HOST = "redis-10008.c274.us-east-1-3.ec2.cloud.redislabs.com"
REDIS_PORT = "10008"
REDIS_PASS = "mswkHhmWSphs7DCAUB4D9BtRtAfYws00"

class RedisConnection(object):

    @staticmethod
    def get_client():
        return redis.Redis(host=REDIS_HOST,
                           port=REDIS_PORT,
                           password=REDIS_PASS,
                           decode_responses=True
                          )

In [21]:
redis = RedisConnection.get_client()
redis.ping()

True

In [175]:
redis.flushdb()

True

In [69]:
# step 1: Redis Functions

def set_hash(key, properties):
        """
        Set the value stored at key to be a hash containing key/value
        """
        
        return redis.hset(key, mapping=properties)

properties = {
    "firstname": "Jane",
    "lastname": "Doe",
    "yearjoined": "2008",
    "vacationdays": "10"
}

set_hash('mykey', properties)

4

In [70]:
def get_hash_field(key, field_name):
    """
    Get the value of an individual field stored in the hash
    at key, returning it as a String.

    :param key: Redis key name.
    :param field_name: Name of the hash field to get value of.
    :return: The value stored at fieldName
    """
    return redis.hget(key, field_name)

def get_hash(key):
    """
    Get the hash stored at key, returning it as a Map.

    :param key: Redis key name.
    :return: Dictionary containing hash field name/value pairs.
    """

    return redis.hgetall(key)

In [108]:
redis.flushdb()

True

# Step 1: Preparation

In [173]:
path = "/Users/brandonamos/Documents/repos/redis-projects/redis/data/breweries.csv"
df = pd.read_csv(path, sep=";")
df.head(2)

,id,breweries,address1,address2,city,state,code,country,phone,website,filepath,descript,last_mod,coordinates
0,117,Big Hole Brewing,401 East Main Street,NaN,Belgrade,Montana,59714,United States,1-800-793-6656,NaN,NaN,NaN,2010-07-22T22:00:20+02:00,"45.7736,-111.172"
1,122,Big Sky Brewing,5417 Trumpeter Way,NaN,Missoula,Montana,59808,United States,1-800-559-2774,NaN,NaN,NaN,2010-07-22T22:00:20+02:00,"46.9223,-114.073"


In [176]:
path = "/Users/brandonamos/Documents/repos/redis-projects/redis/data/worldcities.csv"
df = pd.read_csv(path, sep=",")
df.tail(2)

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
15491,Nordvik,Nordvik,74.0165,111.5100,Russia,RU,RUS,Krasnoyarskiy Kray,NaN,0.0,1643587468
15492,Ennadai,Ennadai,61.1333,-100.8833,Canada,CA,CAN,Nunavut,NaN,0.0,1124019423


In [177]:
dfdict = df.to_dict('records')
dfdict[0]

{'city': 'Tokyo',
 'city_ascii': 'Tokyo',
 'lat': 35.685,
 'lng': 139.7514,
 'country': 'Japan',
 'iso2': 'JP',
 'iso3': 'JPN',
 'admin_name': 'Tōkyō',
 'capital': 'primary',
 'population': 35676000.0,
 'id': 1392685764}

In [178]:
# create location field
for i in dfdict:
    i['location'] = str(str(i['lat'])+str(",")+str(i['lng']))

In [179]:
dfdict[3]

{'city': 'Mumbai',
 'city_ascii': 'Mumbai',
 'lat': 19.017,
 'lng': 72.857,
 'country': 'India',
 'iso2': 'IN',
 'iso3': 'IND',
 'admin_name': 'Mahārāshtra',
 'capital': 'admin',
 'population': 18978000.0,
 'id': 1356226629,
 'location': '19.017,72.857'}

In [180]:
pipe = redis.pipeline(transaction=False)
counter = 0
for i in dfdict:
    counter+=1
    
    key = i['id']
    remove_key = i.pop("id", None)
    properties = i
    #set_hash('ct:'+str(key), properties)
    pipe.hset('ct:'+str(key), mapping=properties)
    if counter % 500 == 0:
        pipe.execute()
        #print(counter)
pipe.execute()

[11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,


In [124]:
# for i in dfdict:
#     key = i['id']
#     remove_key = i.pop("id", None)
#     properties = i
#     set_hash('ct:'+str(key), properties)

In [187]:
redis.hgetall('ct:1840014473')

{'city': 'Oak Ridge',
 'city_ascii': 'Oak Ridge',
 'lat': '35.9638',
 'lng': '-84.2938',
 'country': 'United States',
 'iso2': 'US',
 'iso3': 'USA',
 'admin_name': 'Tennessee',
 'capital': 'nan',
 'population': '29096.0',
 'location': '35.9638,-84.2938'}

# Search for City

In [183]:
from redis.commands.search.field import GeoField, NumericField, TextField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query, NumericFilter

In [ ]:
def __init__(self, index_name):
    self.index_name = index_name
    super().__init__()

def create_property_management_schema(self):
    """
    Create a property management schema that will
    index all hashes starting with "property:"

    The hash has many fields, but we only need to index the following:
    - property_name
    - property_description (should have a lower weight than the name)
    - bedrooms (number of bedrooms; should be sortable)
    - location
    """
    # >>> START CODING CHALLENGE <<<
    self.redis.ft(self.index_name).create_index((
        TextField('property_name', weight=1.0),
        TextField('property_description', weight=0.8),
        NumericField('bedrooms'),
        GeoField('location')),
        definition=IndexDefinition(prefix=['property:'], index_type=IndexType.HASH))
    # >>> START NO ANSWERS <<<
    """
    pass  # TODO implement
    """

In [186]:
index_name = "idx:city"
redis.ft(index_name).create_index((
    TagField('city'),
    TagField('country'),
    TagField('iso2'),
    TagField('iso3'),
    TextField('admin_name'),
    NumericField('population'),
    GeoField('location')),
    definition=IndexDefinition(prefix='ct:', index_type=IndexType.HASH))

'OK'

In [ ]:
index_name = "idx:city"

search_term = "Belo Horizonte"

In [ ]:
# query = Query(search_term).add_filter(NumericFilter('bedrooms', min_bedrooms, max_bedrooms))
# result = redis.ft(index_name).search(query)
# result.docs

# find city

In [197]:
redis.execute_command('FT.SEARCH','idx:city',"@city:{Belo Horizonte}")

[1,
 'ct:1076967355',
 ['city',
  'Belo Horizonte',
  'city_ascii',
  'Belo Horizonte',
  'lat',
  '-19.915',
  'lng',
  '-43.915',
  'country',
  'Brazil',
  'iso2',
  'BR',
  'iso3',
  'BRA',
  'admin_name',
  'Minas Gerais',
  'capital',
  'admin',
  'population',
  '5575000.0',
  'location',
  '-19.915,-43.915']]

In [205]:
redis.execute_command('FT.SEARCH','idx:city',"@location:[-19.915,-43.915 100 km]")

[5,
 'ct:1076863850',
 ['city',
  'Sete Lagoas',
  'city_ascii',
  'Sete Lagoas',
  'lat',
  '-19.4496',
  'lng',
  '-44.25',
  'country',
  'Brazil',
  'iso2',
  'BR',
  'iso3',
  'BRA',
  'admin_name',
  'Minas Gerais',
  'capital',
  'nan',
  'population',
  '201334.0',
  'location',
  '-19.4496,-44.25'],
 'ct:1076456009',
 ['city',
  'Conselheiro Lafaiete',
  'city_ascii',
  'Conselheiro Lafaiete',
  'lat',
  '-20.67',
  'lng',
  '-43.79',
  'country',
  'Brazil',
  'iso2',
  'BR',
  'iso3',
  'BRA',
  'admin_name',
  'Minas Gerais',
  'capital',
  'nan',
  'population',
  '111596.0',
  'location',
  '-20.67,-43.79'],
 'ct:1076876220',
 ['city',
  'Nova Lima',
  'city_ascii',
  'Nova Lima',
  'lat',
  '-19.98',
  'lng',
  '-43.85',
  'country',
  'Brazil',
  'iso2',
  'BR',
  'iso3',
  'BRA',
  'admin_name',
  'Minas Gerais',
  'capital',
  'nan',
  'population',
  '88399.0',
  'location',
  '-19.98,-43.85'],
 'ct:1076967355',
 ['city',
  'Belo Horizonte',
  'city_ascii',
  'Belo H

# add breweries

In [198]:
path = "/Users/brandonamos/Documents/repos/redis-projects/redis/data/breweries.csv"
df = pd.read_csv(path, sep=";")
df.head(2)

,id,breweries,address1,address2,city,state,code,country,phone,website,filepath,descript,last_mod,coordinates
0,117,Big Hole Brewing,401 East Main Street,NaN,Belgrade,Montana,59714,United States,1-800-793-6656,NaN,NaN,NaN,2010-07-22T22:00:20+02:00,"45.7736,-111.172"
1,122,Big Sky Brewing,5417 Trumpeter Way,NaN,Missoula,Montana,59808,United States,1-800-559-2774,NaN,NaN,NaN,2010-07-22T22:00:20+02:00,"46.9223,-114.073"


In [199]:
dfdict = df.to_dict('records')
dfdict[0]

{'id': 117,
 'breweries': 'Big Hole Brewing',
 'address1': '401 East Main Street',
 'address2': nan,
 'city': 'Belgrade',
 'state': 'Montana',
 'code': '59714',
 'country': 'United States',
 'phone': '1-800-793-6656',
 'website': nan,
 'filepath': nan,
 'descript': nan,
 'last_mod': '2010-07-22T22:00:20+02:00',
 'coordinates': '45.7736,-111.172'}

In [200]:
pipe = redis.pipeline(transaction=False)
counter = 0
for i in dfdict:
    counter+=1
    
    key = i['id']
    remove_key = i.pop("id", None)
    properties = i
    #set_hash('ct:'+str(key), properties)
    pipe.hset('brw:'+str(key), mapping=properties)
    if counter % 500 == 0:
        pipe.execute()
        #print(counter)
pipe.execute()

[13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 0,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 0,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 1

In [201]:
redis.hgetall('brw:1034')

{'breweries': 'R&B; Brewing',
 'address1': '54 East Fourth Avenue',
 'address2': 'nan',
 'city': 'Vancouver',
 'state': 'British Columbia',
 'code': '0',
 'country': 'Canada',
 'phone': '1-604-874-2537',
 'website': 'nan',
 'filepath': 'nan',
 'descript': 'nan',
 'last_mod': '2010-07-22T22:00:20+02:00',
 'coordinates': '49.2673,-123.103'}

In [202]:
index_name = "idx:brw"
redis.ft(index_name).create_index((
    TagField('address1'),
    TagField('city'),
    TagField('state'),
    TagField('country'),
    TextField('breweries'),
    GeoField('coordinates')),
    definition=IndexDefinition(prefix='brw:', index_type=IndexType.HASH))

'OK'

In [209]:
#redis.execute_command('FT.SEARCH','idx:brw',"@coordinates:[49.2673,-123.103 100 km]")
redis.execute_command('FT.SEARCH','idx:brw',"@city:{london}")

[4,
 'brw:563',
 ['breweries',
  'Fuller, Smith & Turner PBC',
  'address1',
  'Chiswick Lane South',
  'address2',
  'nan',
  'city',
  'London',
  'state',
  'London',
  'code',
  'nan',
  'country',
  'United Kingdom',
  'phone',
  '44-(0208)-996-2000',
  'website',
  'http://www.fullers.co.uk/',
  'filepath',
  'nan',
  'descript',
  'nan',
  'last_mod',
  '2010-07-22T22:00:20+02:00',
  'coordinates',
  '51.4877,-0.2498'],
 'brw:762',
 ['breweries',
  'Labatt Ontario Breweries',
  'address1',
  '150 Simcoe Street',
  'address2',
  'nan',
  'city',
  'London',
  'state',
  'Ontario',
  'code',
  '0',
  'country',
  'Canada',
  'phone',
  '1-519-663-5050',
  'website',
  'nan',
  'filepath',
  'nan',
  'descript',
  'nan',
  'last_mod',
  '2010-07-22T22:00:20+02:00',
  'coordinates',
  '42.9778,-81.2467'],
 'brw:1342',
 ['breweries',
  'Watney Brewery',
  'address1',
  'nan',
  'address2',
  'nan',
  'city',
  'London',
  'state',
  'London',
  'code',
  'nan',
  'country',
  'United